# Importing packages and datasets.

In [22]:
import pandas
import numpy

import altair

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')
base_file_path = "/content/drive/MyDrive/"

Mounted at /content/drive


In [ ]:
base_file_path = "../data/processed/"

In [3]:
dataFrame = pandas.read_csv(base_file_path + "cleaned.csv")
print(dataFrame.shape)

(2895, 30)


In [35]:
normalisedDataFrame = pandas.read_csv(base_file_path + "normalised.csv")
print(normalisedDataFrame.shape)

(2895, 30)


In [32]:
print(dataFrame.columns)

Index(['avgAnnCount', 'avgDeathsPerYear', 'TARGET_deathRate', 'incidenceRate',
       'medIncome', 'popEst2015', 'povertyPercent', 'studyPerCap', 'MedianAge',
       'MedianAgeMale', 'MedianAgeFemale', 'AvgHouseholdSize',
       'PercentMarried', 'PctNoHS18_24', 'PctHS18_24', 'PctBachDeg18_24',
       'PctHS25_Over', 'PctBachDeg25_Over', 'PctEmployed16_Over',
       'PctUnemployed16_Over', 'PctPrivateCoverage', 'PctEmpPrivCoverage',
       'PctPublicCoverage', 'PctPublicCoverageAlone', 'PctWhite', 'PctBlack',
       'PctAsian', 'PctOtherRace', 'PctMarriedHouseholds', 'BirthRate'],
      dtype='object')


In [8]:
possibly_related_fields = [
                           "medIncome", "povertyPercent", "PctUnemployed16_Over", "PctEmployed16_Over",
                           "PctHS25_Over", "PctBachDeg25_Over",
                           "PctPrivateCoverage", "PctEmpPrivCoverage", "PctPublicCoverage", "PctPublicCoverageAlone",
                           "PctMarriedHouseholds", "PercentMarried"
                           ]

In [33]:
X = dataFrame[possibly_related_fields + ["TARGET_deathRate"]].copy()
y = X.pop("TARGET_deathRate")

train, test = train_test_split(dataFrame, test_size=0.1727, random_state=42)
df_train, df_test = train_test_split(X, test_size=0.1727, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.1727, random_state=42)

print(df_train.shape, "\t", df_test.shape)
print(y_train.shape, "\t", y_test.shape)

(2395, 12) 	 (500, 12)
(2395,) 	 (500,)


In [40]:
X = normalisedDataFrame[possibly_related_fields + ["TARGET_deathRate"]].copy()
y = X.pop("TARGET_deathRate")

train, test = train_test_split(dataFrame, test_size=0.1727, random_state=42)
df_train, df_test = train_test_split(X, test_size=0.1727, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.1727, random_state=42)

print(df_train.shape, "\t", df_test.shape)
print(y_train.shape, "\t", y_test.shape)

(2395, 12) 	 (500, 12)
(2395,) 	 (500,)


# Baseline Performance.

In [41]:
y_mean = y.mean()

y_base = numpy.full((y_test.shape[0], ), y_mean)
(y_base - y_test).sum()

mean_squared_error(y_test, y_base, squared=True)

712.9935696334397

# Training and Testing.

In [ ]:
def results(column_name, test_type):

	reg = LinearRegression().fit(df_train, y_train)

	mse = 0.0
	prediction_line = altair.Chart()
	actual_line = altair.Chart()
	line_chart = altair.Chart()

	if test_type == "test":
		y_preds = reg.predict(df_test)
		prediction_line = altair.Chart(pandas.DataFrame({'x': y_test, 'y': y_preds})).mark_line().encode(
      		x='x',
      		y='y',
			color='blue'
    	)
		actual_line = altair.Chart(pandas.DataFrame({'x': y_test, 'y': y_test})).mark_line().encode(
      		x='x',
      		y='y',
			color='orange'
    	)
		mse = mean_squared_error(y_test, y_preds, squared=True)

	elif test_type == "train":
		prediction_line = altair.Chart(pandas.DataFrame({'x': y_train, 'y': y_preds})).mark_line().encode(
      		x='x',
      		y='y',
			color='blue'
    	)
		actual_line = altair.Chart(pandas.DataFrame({'x': y_train, 'y': y_train})).mark_line().encode(
      		x='x',
      		y='y',
			  color='orange'
    	)
		mse = mean_squared_error(y_train, y_preds, squared=True)

	return (actual_line, prediction_line, mse)

In [47]:
reg = LinearRegression()
reg.fit(df_train, y_train)
y_preds = reg.predict(df_test)
print(mean_squared_error(y_test, y_preds, squared=True))

438.22993820775264


In [45]:
reg = LinearRegression()
reg.fit(df_train, y_train)
y_preds = reg.predict(df_train)
print(mean_squared_error(y_train, y_preds, squared=True))

489.20877939748425


In [48]:
prediction_line = altair.Chart(pandas.DataFrame({'x': y_test, 'y': y_preds})).mark_line().encode(
      x='x',
      y='y'
    )
actual_line = altair.Chart(pandas.DataFrame({'x': y_test, 'y': y_test})).mark_line().encode(
      x='x',
      y='y'
    )

prediction_line + actual_line

alt.LayerChart(...)